## Geo data

In [1]:
import json

with open('sfpddistricts.geojson.json') as data_file:    
    data = json.load(data_file)

In [2]:
coordinates = []
for index in xrange(10):
    if index == 1: coordinates.append(data['features'][index]['geometry']['coordinates'][0][0])
    else: coordinates.append(data['features'][index]['geometry']['coordinates'][0])

In [3]:
dist_lat = {}
latitudes = []
dist_lon = {}
longitudes = []

for i in range(10):
    longitudes = []
    for j in range(len(coordinates[i])):
        longitudes.append(coordinates[i][j][0])
    dist_lon[i] = longitudes    
    
for i2 in range(10):
    latitudes = []
    for j2 in range(len(coordinates[i2])):
        latitudes.append(coordinates[i2][j2][1])
    dist_lat[i2] = latitudes

In [4]:
def find_max(dic):
    
    max_ini = None
    for key, values in dic.iteritems():
        max_val = max(values)
        
        if max_val > max_ini:
            max_ini = max_val
            
    return max_ini

max_lon = find_max(dist_lon)
max_lat = find_max(dist_lat)

print "max lat: %f" % max_lat
print "max lon: %f" % max_lon

max lat: 37.811575
max lon: -122.356983


In [5]:
def find_min(dic):
    
    min_ini = 0
    for key, values in dic.iteritems():
        min_val = min(values)

        if min_val < min_ini:
            min_ini = min_val
            
    return min_ini

min_lon = find_min(dist_lon)

print "min lon: %f" % min_lon

min lon: -122.514949


## Data for visualization

In [6]:
import pandas as pd

def readfile(filename):
    
    data = pd.read_csv(filename)
    return data

filename = 'SFPD_Incidents_-_from_1_January_2003.csv'
#read file
dataset = readfile(filename)

In [7]:
#extract year from date column in the CSV file and only keep them from 2003 to 2015 (full years in the database)
def filteryears(dataset):
    
    #Create a new column with the extracted years from the date column in the CSV file
    dataset['Year'] = [int(date.split("/")[-1]) for date in dataset['Date']]
    #filter the dataset. Just keep the rows that do not belong to year 2016.
    dataset = dataset[dataset['Year'] != 2016]

    return dataset

df_15 = filteryears(dataset)

In [8]:
import geoplotlib
from geoplotlib.utils import BoundingBox

def geo_plot(geodata):
    """
    Plot given coordinate input
    """

    # bounding box on the minima and maxima of the data
    geoplotlib.set_bbox(
        BoundingBox(
            max(geodata['lat']), 
            max(geodata['lon']), 
            min(geodata['lat']), 
            min(geodata['lon'])
        ));
    
    # kernel density estimation visualization
    geoplotlib.kde(geodata, bw=5, cut_below=1e-3, cmap='hot', alpha=170);
    geoplotlib.tiles_provider('mapquest');
    
    # disregard outliers seen as: 90°N 120.5°W and catch only category prostition
include = (df_15.Y < 40) & (df_15.Category == 'PROSTITUTION')

# index geodata
geodata_prostitution = {"lat": df_15.loc[include].Y.tolist(),
                        "lon": df_15.loc[include].X.tolist()}

geo_plot(geodata_prostitution)
geoplotlib.inline();

('smallest non-zero count', 1.4329573088768091e-09)
('max count:', 17.062930260139836)


In [9]:
from sklearn import cluster

def plot_clusters(n_clusters_in, center_only=False):
    print "K={}".format(n_clusters_in)
    coords = ['X','Y']
    X_train = df_15.loc[include][coords]
    cluster_mdl = cluster.KMeans(n_clusters=n_clusters_in)
    k_means = cluster_mdl.fit(X_train)

    centroids = {"lat": k_means.cluster_centers_[:,1],
                 "lon": k_means.cluster_centers_[:,0]}
    
    #get cluster labels
    labels = k_means.labels_
    
    # plot the centers of prostituion together with overall data
    if not center_only:
        geo_plot(geodata_prostitution)
    geoplotlib.dot(centroids, color ='blue', point_size=10);
    geoplotlib.inline();
    
    return centroids, labels, geodata_prostitution
    
centroids, labels, geodata_prostitution = plot_clusters(n_clusters_in=4)

K=4
('smallest non-zero count', 1.4329573088768091e-09)
('max count:', 17.062930260139836)


In [10]:
df_geo = pd.DataFrame()
df_geo['lat'] = geodata_prostitution['lat']
df_geo['lon'] = geodata_prostitution['lon']
df_geo['label'] = labels

file_name = 'geodata_PROST_4.csv'
df_geo.to_csv(file_name, sep=',', index = False)

In [11]:
#CENTROIDS
df_K2 = pd.DataFrame()
df_K2['lat'] = centroids['lat']
df_K2['lon'] = centroids['lon']

filename = 'K4.csv'
df_K2.to_csv(filename, sep=',', index = False)